In [2]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from PIL import Image

In [3]:
TRAIN_DIR = 'images/train'
TEST_DIR  = 'images/test'

In [4]:
# Belirtilen dizindeki görüntüleri ve etiketleri veri çerçevesine yüklemek için fonksiyon
def createdataframe(dir):  
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:
# Eğitim ve test veri çerçevelerini oluşturma
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)


angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [7]:
# Eğitim ve test veri çerçevelerini oluşturma
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [9]:
# Görüntülerden özellik çıkarma fonksiyonu
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [10]:
from tqdm.notebook import tqdm  

In [11]:
# Görüntülerin özelliklerini çıkar
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

In [12]:
# Görüntülerin özelliklerini çıkar
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
# Özellikleri normalize et
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
# Etiketleri sayısal değerlere çevir ve kategorik hale getir
from sklearn.preprocessing import LabelEncoder

In [15]:
# Etiketleri sayısal değerlere çevir ve kategorik hale getir
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
# Etiketleri sayısal değerlere çevir ve kategorik hale getir
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [17]:
# Etiketleri sayısal değerlere çevir ve kategorik hale getir
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [18]:
# Derin öğrenme modelini tanımla
model = Sequential()
# İkinci katman: 128 filtreli 3x3 konvolüsyonel katman, ReLU aktivasyonu, input_shape (48,48,1)
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
# Maksimum havuzlama katmanı: 2x2 havuzlama penceresi
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropout katmanı: %40 oranında
model.add(Dropout(0.4))

# Üçüncü katman: 256 filtreli 3x3 konvolüsyonel katman, ReLU aktivasyonu
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
# Maksimum havuzlama katmanı: 2x2 havuzlama penceresi
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropout katmanı: %40 oranında
model.add(Dropout(0.4))

# Dördüncü katman: 512 filtreli 3x3 konvolüsyonel katman, ReLU aktivasyonu
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# Maksimum havuzlama katmanı: 2x2 havuzlama penceresi
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropout katmanı: %40 oranında
model.add(Dropout(0.4))

# Beşinci katman: 512 filtreli 3x3 konvolüsyonel katman, ReLU aktivasyonu
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# Maksimum havuzlama katmanı: 2x2 havuzlama penceresi
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropout katmanı: %40 oranında
model.add(Dropout(0.4))

# Tam bağlantılı katmanlar için düzleştirme katmanı
model.add(Flatten())
# Tam bağlantılı katman: 512 nöron, ReLU aktivasyonu
model.add(Dense(512, activation='relu'))
# Dropout katmanı: %40 oranında
model.add(Dropout(0.4))
# Tam bağlantılı katman: 256 nöron, ReLU aktivasyonu
model.add(Dense(256, activation='relu'))
# Dropout katmanı: %30 oranında
model.add(Dropout(0.3))
# Çıkış katmanı: 7 nöron, softmax aktivasyonu (7 sınıf, duygular)
model.add(Dense(7, activation='softmax'))

c:\Users\Monster\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
#Modeli eğit
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 


Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 222s 971ms/step - accuracy: 0.2339 - loss: 1.8387 - val_accuracy: 0.2583 - val_loss: 1.8115
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 188s 832ms/step - accuracy: 0.2460 - loss: 1.8085 - val_accuracy: 0.2893 - val_loss: 1.7368
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 189s 836ms/step - accuracy: 0.2869 - loss: 1.7323 - val_accuracy: 0.3722 - val_loss: 1.6053
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 182s 807ms/step - accuracy: 0.3561 - loss: 1.6089 - val_accuracy: 0.4447 - val_loss: 1.4740
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 185s 818ms/step - accuracy: 0.4152 - loss: 1.5046 - val_accuracy: 0.4737 - val_loss: 1.3642
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 1241s 6s/step - accuracy: 0.4450 - loss: 1.4406 - val_accuracy: 0.4943 - val_loss: 1.3246
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 210s 931ms/step - accuracy: 0.4542 - loss: 1.4115 - val_accuracy: 0.5137 - val_loss: 1.2734
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 219s 969ms/step - accuracy: 0.

KeyboardInterrupt: 

In [ ]:
# Modeli JSON formatında kaydet
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
# Modeli yükle
from keras.models import model_from_json , Sequential

In [ ]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json, custom_objects={'Sequential': Sequential})
model.load_weights("facialemotionmodel.h5")

In [ ]:
# Etiketler
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
# Görüntüden özellik çıkarma fonksiyonu
def ef(image):
    img = load_img(image, color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0
    

In [ ]:
# Örnek bir görüntü üzerinde tahmin yapma ve sonucu yazdırma
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')